In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', cache=False)

In [5]:
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')

In [7]:
print(X.shape)
print(y.shape)

(70000, 784)
(70000,)
